In [139]:
import numpy as np
from pathlib import Path
import napari
from tqdm import tqdm
import pandas as pd
from cellpose import models, core, io
import pyclesperanto_prototype as cle 
from skimage.measure import regionprops_table
from utils import list_images, read_image, extract_scaling_metadata

io.logger_setup() # run this to get printing of progress

#Check if notebook has GPU access
if core.use_gpu()==False:
  raise ImportError("No GPU access, change your runtime")

#Load pre-trained Cellpose models
model = models.CellposeModel(gpu=True, model_type="cyto3") 

creating new log file
2026-01-21 15:49:04,954 [INFO] WRITING LOG OUTPUT TO C:\Users\adiez_cmic\.cellpose\run.log
2026-01-21 15:49:04,954 [INFO] 
cellpose version: 	3.1.1.3 
platform:       	win32 
python version: 	3.10.19 
torch version:  	2.5.0
2026-01-21 15:49:04,954 [INFO] ** TORCH CUDA version installed and working. **
2026-01-21 15:49:04,954 [INFO] >> cyto3 << model set to be used
2026-01-21 15:49:04,954 [INFO] ** TORCH CUDA version installed and working. **
2026-01-21 15:49:04,954 [INFO] >>>> using GPU (CUDA)
2026-01-21 15:49:05,080 [INFO] >>>> loading model C:\Users\adiez_cmic\.cellpose\models\cyto3
2026-01-21 15:49:05,313 [INFO] >>>> model diam_mean =  30.000 (ROIs rescaled to this size during training)


In [140]:
markers = [("Occludin_RFP", 0, "membrane"), ("Claudin_FITC", 1, "membrane")]

In [141]:
# Copy the path where your images are stored, you can use absolute or relative paths to point at other disk locations
#directory_path = Path(r"\\forskning.it.ntnu.no\ntnu\mh\ikom\cmic_konfokal\lusie.f.kuraas\PhD\Nikon Spinning Disc\20260114_T7_2microns")
directory_path = Path("./raw_data")

# Iterate through the .czi and .nd2 files in the directory
images = list_images(directory_path)

# Image size reduction (downsampling) to improve processing times (slicing, not lossless compression)
slicing_factor_xy = None # Use 2 or 4 for downsampling in xy (None for lossless)

images

['raw_data\\B2.nd2']

In [142]:
# Initialize Napari Viewer
viewer = napari.Viewer(ndisplay=2)

In [143]:
# Explore a different image (0 defines the first image in the directory)
image = images[0]

# Read image, apply slicing if needed and return filename and img as a np array
img, filename = read_image(image, slicing_factor_xy)

# Extract well_id from filename
well_id = filename.split("_")[0]

# Extract x,y,z scaling from .nd2 file metadata in order feed the Z pixel size / XY pixel size ratio into Cellpose
pixel_size_x, pixel_size_y, voxel_size_z = extract_scaling_metadata(image)

assert pixel_size_x == pixel_size_y
z_to_xy_ratio = voxel_size_z / pixel_size_x
print(z_to_xy_ratio)


Image analyzed: B2
Pixel size: 0.663 µm x 0.663 µm
Voxel (Z-step) size: 2.000 µm
3.017451031446116


In [144]:
#TODO: Extract number of multipos and loop through that range (store position for data extraction)
# Open one of the multipositions in the img file
img = img[3]
# Input shape for next step has to be C, Z, Y, X
img = img.transpose(1, 0, 2, 3)

# Keep only 3 meaningful input fluorescence channels to mimick Cellpose GUI normalization
# 0 membrane marker, 2 nuclei, 3 Cellmask
cellpose_input = img[[0, 2, 3],:,:,:]
img.shape

(5, 21, 1360, 1360)

In [145]:
masks, flows, styles = model.eval(
    cellpose_input,                # numpy array
    channels=[3, 2],          # channels=[cyto_chan, nuclear_chan]
    diameter=20,              # in pixels (XY), checked with GUI
    do_3D=True,
    anisotropy=z_to_xy_ratio,           # Z pixel size / XY pixel size
    normalize=True,
    flow_threshold=0.4,
    cellprob_threshold=0.0,
    min_size=15,
)
del cellpose_input

2026-01-21 15:49:08,227 [INFO] multi-stack tiff read in as having 21 planes 3 channels
2026-01-21 15:49:08,451 [INFO] resizing 3D image with rescale=1.50 and anisotropy=3.017451031446116
2026-01-21 15:49:09,770 [INFO] running YX: 95 planes of size (2040, 2040)
2026-01-21 15:49:46,784 [INFO] 100%|##########| 95/95 [00:37<00:00,  2.57it/s]
2026-01-21 15:49:49,415 [INFO] running ZY: 2040 planes of size (95, 2040)
2026-01-21 15:50:24,819 [INFO] 100%|##########| 2040/2040 [00:35<00:00, 57.62it/s]
2026-01-21 15:50:26,308 [INFO] running ZX: 2040 planes of size (95, 2040)
2026-01-21 15:51:09,500 [INFO] 100%|##########| 2040/2040 [00:43<00:00, 47.23it/s]
2026-01-21 15:51:14,519 [INFO] resizing 3D flows and cellprob to original image size
2026-01-21 15:51:15,738 [INFO] network run in 127.29s
2026-01-21 15:51:18,093 [INFO] masks created in 1.77s


In [146]:
viewer.add_image(img)
viewer.add_labels(masks)

<Labels layer 'masks' at 0x18cdde7f700>

In [147]:
markers

# Create a dictionary containing all image descriptors
#TODO: Add multiposition index during BP
descriptor_dict = {
            "filename": filename,
            "well_id": well_id,
            }

In [148]:
props_list = []

# Loop through markers and extract 
for marker_name, ch_nr, location in markers:
    print(f"Analyzing channel: {marker_name}")

    props = regionprops_table(label_image=masks,
                            intensity_image=img[ch_nr],
                            properties=[
                                "label",
                                "area",
                                "intensity_mean",
                                "intensity_min",
                                "intensity_max",
                                "intensity_std",
                            ],
                        )
    
    # Convert to dataframe
    props_df = pd.DataFrame(props)

    # Rename intensity_mean column to indicate the specific image
    prefix = f"{location}_{marker_name}"

    rename_map = {
        "intensity_mean": f"{prefix}_mean_int", # concentration proxy
        "intensity_min":  f"{prefix}_min_int",
        "intensity_max":  f"{prefix}_max_int",
        "intensity_std":  f"{prefix}_std_int",
    }

    props_df.rename(columns=rename_map, inplace=True)

    # Max / mean ratio (puncta vs diffuse signal)
    props_df[f"{prefix}_max_mean_ratio"] = (props_df[f"{prefix}_max_int"] /props_df[f"{prefix}_mean_int"])
    # Total marker content per cell
    props_df[f"{prefix}_sum_int"] = (props_df[f"{prefix}_mean_int"] * props_df["area"])

    # Append each props_df to props_list
    props_list.append(props_df)

    # Initialize the df with the first df in the list
    props_df = props_list[0]
    # Start looping from the second df in the list
    for df in props_list[1:]:
        props_df = props_df.merge(df, on="label")

# Add each key-value pair from descriptor_dict to props_df at the specified position
insertion_position = 0    
for key, value in descriptor_dict.items():
    props_df.insert(insertion_position, key, value)
    insertion_position += 1  # Increment position to maintain the order of keys in descriptor_dict

Analyzing channel: Occludin_RFP
Analyzing channel: Claudin_FITC


In [149]:
props_df

,filename,well_id,label,area_x,membrane_Occludin_RFP_mean_int,membrane_Occludin_RFP_min_int,membrane_Occludin_RFP_max_int,membrane_Occludin_RFP_std_int,membrane_Occludin_RFP_max_mean_ratio,membrane_Occludin_RFP_sum_int,area_y,membrane_Claudin_FITC_mean_int,membrane_Claudin_FITC_min_int,membrane_Claudin_FITC_max_int,membrane_Claudin_FITC_std_int,membrane_Claudin_FITC_max_mean_ratio,membrane_Claudin_FITC_sum_int
0,B2,B2,1,3424.0,277.589661,136.0,983.0,80.776562,3.541198,950467.0,3424.0,532.482185,233.0,1256.0,164.412146,2.358764,1823219.0
1,B2,B2,2,3256.0,204.658784,105.0,1087.0,73.148733,5.311279,666369.0,3256.0,227.188575,110.0,708.0,72.642705,3.116354,739726.0
2,B2,B2,3,4189.0,249.182144,132.0,904.0,73.875658,3.627868,1043824.0,4189.0,495.893053,182.0,1701.0,178.672270,3.430175,2077296.0
3,B2,B2,4,3213.0,302.078120,136.0,930.0,87.169359,3.078674,970577.0,3213.0,534.435730,167.0,1246.0,175.541747,2.331431,1717142.0
4,B2,B2,5,1523.0,172.711753,108.0,363.0,43.761864,2.101768,263040.0,1523.0,336.382797,91.0,1959.0,269.876845,5.823722,512311.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1293,B2,B2,1294,91.0,152.714286,121.0,195.0,16.148545,1.276894,13897.0,91.0,374.241758,249.0,543.0,67.072266,1.450934,34056.0
1294,B2,B2,1295,33.0,160.333333,129.0,201.0,17.862550,1.253638,5291.0,33.0,463.272727,286.0,782.0,122.479008,1.687991,15288.0
1295,B2,B2,1296,34.0,211.911765,146.0,281.0,34.715963,1.326024,7205.0,34.0,579.764706,419.0,700.0,65.448704,1.207386,19712.0
1296,B2,B2,1297,41.0,194.463415,147.0,254.0,28.515377,1.306158,7973.0,41.0,597.024390,432.0,806.0,78.999687,1.350029,24478.0


In [150]:
import numpy as np
from scipy.ndimage import distance_transform_edt
from skimage.filters import threshold_otsu
from skimage.segmentation import watershed
from skimage.morphology import remove_small_objects


In [151]:
# --- Build organoid MIP ---
nuclei_mip = np.max(img[2], axis=0)
cellmask_mip = np.max(img[3], axis=0)

mip = nuclei_mip + cellmask_mip
viewer.add_image(mip, name="organoid_MIP")


# --- Smooth large structures ---
mip_blurred = cle.gaussian_blur(
    mip,
    sigma_x=5,
    sigma_y=5,
)
viewer.add_image(mip_blurred, name="organoid_MIP_blurred")


# --- Voronoi-Otsu: coarse organoid separation ---
organoid_seeds = cle.voronoi_otsu_labeling(
    mip_blurred,
    spot_sigma=50,     # controls seed spacing (bigger = fewer organoids)
    outline_sigma=10,  # boundary smoothness
)
viewer.add_labels(organoid_seeds, name="organoids_voronoi")


# --- Otsu mask to constrain watershed ---
organoid_mask = cle.threshold_otsu(mip_blurred)
viewer.add_labels(organoid_mask, name="organoid_mask")


# --- Distance transform inside organoid mask ---
# Ensure binary mask
organoid_mask_np = cle.pull(organoid_mask) > 0

# --- Convert data to NumPy ---
mip_blurred_np = cle.pull(mip_blurred)
organoid_seeds_np = cle.pull(organoid_seeds)

# --- Otsu mask (CPU) ---
threshold = threshold_otsu(mip_blurred_np)
organoid_mask_np = mip_blurred_np > threshold

viewer.add_labels(organoid_mask_np, name="organoid_mask")

# --- Distance transform (CPU) ---
distance_map_np = distance_transform_edt(organoid_mask_np)
viewer.add_image(distance_map_np, name="distance_map")

# --- Marker-controlled watershed (CPU) ---
# IMPORTANT: use NEGATIVE distance for watershed
organoid_labels_np = watershed(
    -distance_map_np,
    markers=organoid_seeds_np,
    mask=organoid_mask_np,
)

viewer.add_labels(organoid_labels_np, name="organoids_watershed")

# --- Cleanup ---
organoid_labels_np = remove_small_objects(
    organoid_labels_np,
    min_size=10000,
)

viewer.add_labels(organoid_labels_np, name="organoids_final")


<Labels layer 'organoids_final' at 0x18d2e8c7070>